In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# Define root_path

In [ ]:
# Load calc_df
calc_df.shape

In [ ]:
# Step 1: Filter columns
score_columns = [col for col in calc_df.columns if "score" in col]

# Step 2: Extract hierarchy information and prepare data
data = []
for col in score_columns:
    parts = col.split('_')
    if col.startswith('bleu'):
        type_ = parts[2]
        guidance = parts[3]
    elif col.startswith('bert'):
        type_ = parts[3]
        guidance = parts[4]
    else:
        type_ = parts[3]
        guidance = parts[4]
    ref_context = "reference" if "reference" in col else "context"
    score_type = parts[0] + '_' + parts[1]  # bleu_score, bert_score_f, cross_encoder_score
    # Calculate the mean value for each column
    mean_score = calc_df[col].mean()
    std_score = calc_df[col].std()
    data.append([type_, guidance, ref_context, score_type, mean_score, std_score])

# Convert the list to a DataFrame
pivot_data = pd.DataFrame(data, columns=['Type', 'Guidance', 'Reference/Context', 'Score Type', 'Mean Score', 'Std Score'])
pivot_data

In [ ]:
# Step 3: Create pivot tables for mean score and standard deviation score
pivot_table_mean = pivot_data.pivot_table(index=['Type', 'Guidance', 'Reference/Context'], columns='Score Type', values='Mean Score', aggfunc='first')
pivot_table_std = pivot_data.pivot_table(index=['Type', 'Guidance', 'Reference/Context'], columns='Score Type', values='Std Score', aggfunc='first')

# Step 4: Combine the two pivot tables
pivot_table_combined = pd.concat([pivot_table_mean, pivot_table_std], axis=1, keys=['Mean Score', 'Std Score'])

# Flatten the columns for better readability
pivot_table_combined.columns = [f'{metric}_{score_type}' for metric, score_type in pivot_table_combined.columns]

pivot_table_combined

In [7]:
# Define excel_path
pivot_data.to_excel(excel_path, index=False)